# Qual-IF-AI: Quality Control of ImmunoFluorescence Images using Artificial Intelligence

### Step 0 : Load functions
#### Step 0 is reading file with all the functions to implement Qual-IF-AI

In [ ]:
%run QC_model_implementation

### Step1: Tessalate image
####  Qual-IF-AI models take as an input images of size 512 X 512.'split tiles' function tessalates input image into smaller tiles of size 512 X 512. Arguments : 1) input_image_path- Path to input images. 2)path_output_tiles- Path to save output tessaleted tiles

In [ ]:
input_image_path=os.path.join(wd,'..','Images/')
path_output_tiles=os.path.join(wd,'..','cut_tiles')

split_tiles(input_image_path,path_output_tiles)

### Step2: Classification 
##### This function predicts probability of having 1)Air bubble 2)External artifact 3)Out of focus areas 4)Tissur folds 5)Antibody aggregations in each splitted tile. For predicting classification label, for each artifact, pre last vector output of the model is projected on a PCA template of images with artifact and images without artifacts. If 8 (n) out of 10 (k) nearest neighbors belong to one category, then that category label is given to the image tile else 'doubt' label is given. Each image tile i Classification label is given based on classification probability and number of  nearest neighbors of model  Arguments: 1) input path to split tiles 2)k:number of nearest neighbors to be accessed 3)n: how many neighbors should have same label 4)path to save output csv. 


In [ ]:
path_output_tiles=os.path.join(wd,'..','cut_tiles')
csv_path=os.path.join(wd,'..','Data','classification_prediction.csv')
k=10
n=8
classification_prediction(path_output_tiles,csv_path,k,n)

### Step3: Segmentation

#### This function generates segmentation mask for every input images which classified as having artifact by the classification module. Arguments 1)artifact_list: Path to csv geberated by the classification module 2)out_dir: -path to the folder for saving output masks 3)img_path: Path to tessalated tiles

In [ ]:
artifact_list=os.path.join(wd,'..','Data','classification_prediction.csv')
out_dir=os.path.join(wd,'..','mask')
img_path=os.path.join(wd,'..','cut_tiles')
segmentation(img_path,out_dir,artifact_list)

### Step4: Stitching mask tiles
####  This step stiches together all the masks from predicted tiles to match the size of original image. Arguments 1)raw_image_path: Path to original images 2)output_path: Path to output folder 3)input_image_path: Path to the folder where segmentation predictions of each image tile is saved

In [ ]:
raw_image_path=out_dir=os.path.join(wd,'..','Images')
output_path=os.path.join(wd,'..','hard_stitch_masks')
input_image_path=os.path.join(wd,'..','mask')


merge_predictions(input_image_path,output_path,raw_image_path)

### Step 5: Generating artifact free images
#### In this final step artifacts from the images are removed using the masks generated previously

In [ ]:
Mask_path=os.path.join(wd,'..','hard_stitch_masks')
Raw_image_path=os.path.join(wd,'..','Images')
output_path=os.path.join(wd,'..','Images_corrected')

artifacts_free_image(Mask_path,Raw_image_path,output_path)

### Wrapper function

#### This function generates all the steps together


In [ ]:
input_image_path=os.path.join(wd,'..','Images/')
output_path=os.path.join(wd,'..','Images_corrected')
QualIFAI(input_image_path,output_path)